## Introduction à la quantization 

Laurent cetinsoy

Les réseaux de neurones prennent beaucoup de place et il peut être difficile de les faire rentrer sur certains dispositifs embarqués. 

Il existe plusieurs méthodes pour réduire la taille et augmenter la vitesse d'executer des réseaux de neurone. Par exemple il y a ce qu'on appelle la quantization et le pruning.

Dans ce notebook on va faire une introduction à la quantization avec la librairie tensorflow lite.


## Quantization post training

Dans un premier temps on va quantifier notre réseau après l'avoir entraîné normalement. 


Entraîner un réseau de neurone convolutionnel simple avec keras pour faire de la classification MNIST (ou un autre dataset simple de votre choix si (vous en avez marre de ce dataset - https://keras.io/api/datasets/)




In [9]:
from google.colab import drive
drive.mount('/content/drive')

dir_path = '/content/drive/MyDrive/ST2IASE/efreiembarque/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
from tensorflow.keras.datasets.mnist import load_data

In [13]:
train,test = load_data()
X_train,y_train = train
X_test,y_test = test

11490434/11490434 [==============================] - 1s 0us/step


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense

In [ ]:
model = Sequential()
model.add(Conv2D(8,kernel_size=(3,3), activation='relu'))
model.add(Conv2D(32,kernel_size=(3,3), activation='relu'))

model.add(Flatten())

model.add(Dense(100,activation='relu'))
# Nous avons 10 classes (0 à 9)
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
X_train_reshaped = X_train.reshape(-1, 28, 28, 1)/255 #MinMaxScaling
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)/255 #MinMaxScaling

In [ ]:
model.fit(X_train_reshaped,y_train,validation_data=(X_test_reshaped, y_test)) #final accuracy must be > 95%

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1875/1875 [==============================] - 84s 45ms/step - loss: 0.1387 - accuracy: 0.9585 - val_loss: 0.0612 - val_accuracy: 0.9795


Afficher le nombre de paramètre du modèle

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 8)         80        
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        2336      
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 100)               1843300   
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 1,846,726
Trainable params: 1,846,726
Non-trainable params: 0
_________________________________________________________________


Sauvegarder votre modèle et afficher la taille du fichier. Si on applique une bête règle de trois, quelle est la taille occupée par paramètre ? 

In [6]:
import joblib

In [ ]:
joblib.dump(model,dir_path+'conv2D.joblib')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......flatten
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-21 20:38:55         2652
variables.h5                                   2023-03-21 20:38:55     22189968
metadata.json                                  2

['/content/drive/MyDrive/ST2IASE/efreiembarque/conv2D.joblib']

In [ ]:
# 1846726 -> 22189968
# nb de parametres -> taille en octet
print("Taille par parametre:",22189968/1846726,"octets")

Taille par parametre: 12.015842090272189 octets


On va maintenant convertir notre modèle keras en modèle tensorflow lite. 

Installer la librairie tensorflow lite créer une instance de la class TFLiteConverter à partir de votre modèle keras


In [ ]:
!pip install tflite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Convertir votre modèle et le sauvegarder dans un fichier nommé model.tflite. Sa taille est-elle plus petite ? 

In [3]:
import tensorflow as tf

In [ ]:
model = joblib.load(dir_path+"conv2D.joblib")
tf.saved_model.save(model, dir_path+'model')

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-21 20:38:54         2652
variables.h5                                   2023-03-21 20:38:54     22189968
metadata.json                                  2023-03-21 20:38:54           64
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......flatten
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
........

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(dir_path+"model") # path to the SavedModel directory
tflite_model = converter.convert()

In [ ]:
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)
%cp /content/model.tflite /content/drive/MyDrive/ST2IASE/efreiembarque

Pour le premier modèle, on a une taille de 21 mo alors que pour le second, on a une taille de 7 mo.

On va maintenant spécifier des optimisations au converter. 

1. Recréer un converter

2. modifier son attribut optimizations pour ajouter une liste d'optimisation avec la valeur tf.lite.Optimize.DEFAULT

3. Relancer la conversion du modèle, sauvegarder le modèle et regarder la taille du fichier généré

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(dir_path+"model") # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [ ]:
with open('model_ptq.tflite', 'wb') as f:
  f.write(tflite_model)
%cp /content/model_ptq.tflite /content/drive/MyDrive/ST2IASE/efreiembarque

On passe de 7 mo à 1.8 mo.

Quelle type  de quantization Optimize.Default, utilise-t-elle ?


Elle utilise une quantification de plage dynamique (réduit l'utilisation de la mémoire, accélération des calculs).

## Quantization aware training 

Dans cette section on va s'intéresser à l'entraînement sensible à la quantification. L'idée est de simuler les effets de la quantification pendant l'entraînement pour que le modèle ajuste les poids afin de tenir ocmpte de la quantification. L'idée est de prendre un modèle déjà entraîné normalement et de le réentraîné en faisant un peu de quantization pendant l'entraînement. 


Reprendre le modèle entraîné sur MNIST


In [ ]:
model = joblib.load(dir_path+"conv2D.joblib")

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-21 20:30:10         2642
variables.h5                                   2023-03-21 20:30:10     22189968
metadata.json                                  2023-03-21 20:30:10           64
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......flatten
.........vars
...metrics
......mean
.........vars
............0
............1
......mean_metric_wrapper
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........2
.........3
.........4
.........5
.........6
........

A l'aide de la fonction quantize de tensorflow_model_optimization, créer une seconde version de votre modèle entraîné nommé qat_model

In [ ]:
!pip install tensorflow_model_optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow_model_optimization as tfmot

In [ ]:
quantize_model = tfmot.quantization.keras.quantize_model
qat_model = quantize_model(model)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Compiler le modèle

In [ ]:
qat_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Afficher le summary du modèle. D'après vous ce modèle est-il quantifié ? 

In [ ]:
qat_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLay  (None, 28, 28, 1)        3         
 er)                                                             
                                                                 
 quant_conv2d (QuantizeWrapp  (None, 26, 26, 8)        99        
 erV2)                                                           
                                                                 
 quant_conv2d_1 (QuantizeWra  (None, 24, 24, 32)       2403      
 pperV2)                                                         
                                                                 
 quant_flatten (QuantizeWrap  (None, 18432)            1         
 perV2)                                                          
                                                                 
 quant_dense (QuantizeWrappe  (None, 100)              1

Réentraîner votre modèle sur un sous ensemble des données (sur une ou deux epochs) et afficher la performance sur le train et test set

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
y_train_reshaped = to_categorical(y_train, 10)
y_test_reshaped = to_categorical(y_test, 10)
y_train_reshaped.shape

(60000, 10)

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
qat_model.fit(X_train_reshaped, y_train_reshaped, epochs=2, validation_data=(X_test_reshaped, y_test_reshaped))
joblib.dump(qat_model,dir_path+"conv2D_qat.joblib")
tf.saved_model.save(qat_model, dir_path+'model_qat')

/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/2
1875/1875 [==============================] - 157s 84ms/step - loss: 0.0443 - accuracy: 0.9863 - val_loss: 0.0366 - val_accuracy: 0.9881
Epoch 2/2
1875/1875 [==============================] - 143s 76ms/step - loss: 0.0212 - accuracy: 0.9936 - val_loss: 0.0559 - val_accuracy: 0.9818
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......quantize_layer
.........vars
............0
............1
............2
......quantize_wrapper_v2
.........layer
............vars
...............0
.........vars
............0
............1
............2
............3
............4
............5
............6
............7
......quantize_wrapper_v2_1
.........layer
............vars
...............0
.........vars
............0
............1
............2
............3
............4
............5
............6
............7
......quantize_wrapper_v2_2
.........layer
............vars
.........vars
............0
......quantize_wrapper_v2_3
.........layer
............vars
...

Convertir votre modèle avec TFLite

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(dir_path+"model_qat") # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

Sauvegarder le modèle QAT et comparer les tailles des modèles

In [ ]:
with open('model_qat.tflite', 'wb') as f:
  f.write(tflite_model)
%cp /content/model_qat.tflite /content/drive/MyDrive/ST2IASE/efreiembarque

Comparer les performances des trois modèles suivants (taille et accuracy) : 
- modèle original
- modèle quantifié avec la post training quantization
- modèle entraîné avec la training aware quantization




OG: 21.2 mo
tflite: 7 mo
ptq: 1.8 mo
qat: 1.8 mo

In [29]:
og = tf.saved_model.load(dir + 'model')
qat = tf.saved_model.load(dir + 'model_qat')

In [32]:
import numpy as np

In [36]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(1)

In [37]:
og_metrics = og.evaluate(test_dataset)
qat_metrics = qat.evaluate(test_dataset)

AttributeError: ignored

In [27]:
og = joblib.load(dir_path + 'conv2D.joblib')

ValueError: ignored

In [24]:
# Charger le modèle tflite
interpreter = tf.lite.Interpreter(model_path=dir_path+"model_qat.tflite")
interpreter.allocate_tensors()

# Récupérer les index des tenseurs d'entrée et de sortie
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Charger les données de test
x_test = X_test.reshape(-1,28,28,1) # Données d'entrée de taille (n_samples, input_shape)
y_test = y_test # Données de sortie de taille (n_samples, output_shape)

# Préparer les tenseurs d'entrée et de sortie
input_data = x_test.astype(input_details[0]['dtype'])
output_data = y_test.astype(output_details[0]['dtype'])

# Évaluer le modèle sur les données de test
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
y_pred = interpreter.get_tensor(output_details[0]['index'])

# Calculer les métriques
accuracy = (y_pred == y_test).mean()
#loss = ... # Calculer la perte en fonction du type de problème (classification, régression, etc.)

print("Accuracy:", accuracy)
#print("Loss:", loss)


ValueError: ignored

In [23]:
X_test[0].reshape(-1,28,28,1).shape

(1, 28, 28, 1)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0d51e245-899d-41d6-b23b-cf3e4bbbc6ea' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>